In [ ]:
#1. rutin1 import module
import pandas as pd
import os
import sys
import warnings
import matplotlib.pyplot as plt

import networkx as nx
from collections import defaultdict

warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(repo_root)

In [ ]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r30_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]
#df_pos = df['pos_sentence_list']

In [ ]:
def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    magnitude1 = sum(a**2 for a in vec1) ** 0.5
    magnitude2 = sum(b**2 for b in vec2) ** 0.5
    if magnitude1 * magnitude2 == 0:
        return 0
    return dot_product / (magnitude1 * magnitude2)

In [ ]:
stopwords_path = os.path.join(repo_root, "notebooks/stopwords_tuning/all_stop_words.txt")
with open(stopwords_path, 'r') as file:
    stopwords = set(file.read().strip().splitlines())

In [ ]:
def textrank(text, num_keywords=5):
    # Tokenize the text
    words = word_tokenize(text)
    
    # Load stopwords for the specified language
    stop_words = stopwords
    
    # Filter out stopwords and punctuation
    words = [word.lower() for word in words if word.lower() not in stop_words and word not in string.punctuation]
    
    # Build a co-occurrence matrix
    co_occurrence = defaultdict(int)
    window_size = 2
    for i in range(len(words) - window_size + 1):
        window = words[i:i+window_size]
        for j in range(window_size):
            for k in range(j+1, window_size):
                w1, w2 = sorted([window[j], window[k]])
                if w1 != w2:
                    co_occurrence[(w1, w2)] += 1
    
    # Build a graph
    G = nx.Graph()
    for (w1, w2), weight in co_occurrence.items():
        
        # Get word embeddings
        vec1 = nlp(w1).vector
        vec2 = nlp(w2).vector

        # Calculate similarity score
        similarity = cosine_similarity(vec1, vec2)

        G.add_edge(w1, w2, weight=weight)
    
    # Compute TextRank scores
    scores = nx.pagerank(G)
    
    # Sort words by scores
    ranked_words = sorted(((score, word) for word, score in scores.items()), reverse=True)
    
    # Extract the top keywords
    keywords = [word for score, word in ranked_words[:num_keywords]]
    
    return keywords
